# Step 2 — Threshold Rule + Complementary Movement

## **Goal**
Build on Step 1 by adding a “backup rule” so that when an agent fails the threshold rule,  
they can still move toward a complementary (different) neighbor before staying or moving to random.

---

## **Key Additions from Step 1**
- **anti-backtrack**: prevent unrealistic ping-pong movement between two nodes, stabilizing the diffusion and making footfall patterns more realistic  
- Add **p_move_below**: if threshold not met, move to different-category node with this probability.  
- Define a clear **stay condition**: if both fail, agent stays or makes a minimal random move within its neighbors.  
- Anchor remains fixed (e.g., node 0) to ensure consistent diffusion tracking.

---

## **Movement Logic Summary**
| Step | Condition | Action |
|------|------------|--------|
| ① | `ratio_same ≥ θ_same` | Move to a *similar* neighbor |
| ② | `ratio_same < θ_same` and random < `p_move_below` | Move to a *different* neighbor (backup rule) |
| ③ | Otherwise | Stay (or random minimal move) |

---

## **Parameter Settings**
| Parameter | Meaning | Example |
|------------|----------|----------|
| `θ_same` | Same-category ratio threshold | 0.6 |
| `p_move_below` | Probability of moving to different node below threshold | 0.4 |
| `p_stay_anchor` | Probability of staying at anchor | 0.3 |
| `steps` | Number of time ticks | 10 |
| `n_agents` | Number of customers | 100 |

---

## **Interpretation**
- When **θ_same ↑**, agents cluster near similar stores.  
- When **p_move_below ↑**, diffusion toward other categories increases.  
- Fixed anchor allows consistent measurement of footfall spread and spillover effects.

---

In [26]:
# --- Notebook setup (self-contained) ---
%load_ext autoreload
%autoreload 2
%matplotlib inline

import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from dataclasses import dataclass
from IPython.display import Image, display, Markdown
import os

def ensure_dir(path: str):
    os.makedirs(path, exist_ok=True)


In [27]:
@dataclass
class Params:
    n_nodes: int = 20
    n_anchors: int = 1
    steps: int = 50
    theta_same: float = 0.6
    p_edge: float = 0.2
    n_agents: int = 200
    p_stay_anchor: float = 0.2
    p_move_below: float = 0.5
    anti_backtrack: bool = True
    backtrack_penalty: float = 0.5
    seed: int = 1
    count_t0: bool = True
    init_spread_ratio: float = 0.30
    agent_marker_size: int = 90
    jitter: float = 0.012
    interval_ms: int = 200

def build_mall(params: Params):
    rng = np.random.default_rng(params.seed)
    G = nx.erdos_renyi_graph(params.n_nodes, params.p_edge, seed=params.seed)
    anchors = {0}
    for n in G.nodes:
        if n in anchors:
            G.nodes[n]["role"] = "anchor"
            G.nodes[n]["category"] = "similar"
            G.nodes[n]["A"] = 3
        else:
            G.nodes[n]["role"] = "tenant"
            G.nodes[n]["category"] = rng.choice(["similar", "different"])
            G.nodes[n]["A"] = 1
    return G, anchors

def is_complement(a: str, b: str) -> bool:
    return a != b

def simulate(params: Params):
    rng = np.random.default_rng(params.seed)
    G, anchors = build_mall(params)
    A0 = next(iter(anchors))
    footfall = np.zeros(params.n_nodes, dtype=int)

    customers = np.full(params.n_agents, A0, dtype=int)
    neighbors_A0 = list(G.neighbors(A0)) or [A0]
    k = int(params.init_spread_ratio * params.n_agents)
    if k > 0:
        idx = rng.choice(params.n_agents, size=k, replace=False)
        customers[idx] = rng.choice(neighbors_A0, size=k, replace=True)

    prev_pos = customers.copy()
    traces = []

    if params.count_t0:
        footfall += np.bincount(customers, minlength=params.n_nodes)
        traces.append(customers.copy())

    for _ in range(params.steps):
        for i in range(params.n_agents):
            here = customers[i]
            neighbors = list(G.neighbors(here)) or [here]
            curr_cat = G.nodes[here]["category"]

            same_neighbors = [v for v in neighbors if G.nodes[v]["category"] == curr_cat]
            ratio_same = len(same_neighbors) / len(neighbors)

            if G.nodes[here]["role"] == "anchor" and rng.random() < params.p_stay_anchor:
                nxt = here
            elif same_neighbors and ratio_same >= params.theta_same:
                nxt = int(rng.choice(same_neighbors))
            else:
                comp_neighbors = [v for v in neighbors if is_complement(curr_cat, G.nodes[v]["category"])]
                if rng.random() < params.p_move_below:
                    pool = comp_neighbors if comp_neighbors else neighbors
                    nxt = int(rng.choice(pool))
                else:
                    nxt = here

            if params.anti_backtrack and nxt == prev_pos[i] and rng.random() < params.backtrack_penalty:
                nxt = here

            prev_pos[i] = here
            customers[i] = nxt

        footfall += np.bincount(customers, minlength=params.n_nodes)
        traces.append(customers.copy())

    df = pd.DataFrame({
        "node": np.arange(params.n_nodes),
        "role": [G.nodes[n]["role"] for n in G.nodes],
        "category": [G.nodes[n]["category"] for n in G.nodes],
        "footfall": footfall
    })

    ensure_dir("data/outputs")
    df.to_csv("data/outputs/step2_visible.csv", index=False)
    print("[Saved] data/outputs/step2_visible.csv")

    class SimResult: ...
    sim = SimResult()
    sim.mall_graph = G
    sim.traces = traces
    sim.p = params
    return df, sim, A0

def animate_movement(sim, filename="figs/step2_visible.gif", pos=None, interval=400):
    from matplotlib.animation import FuncAnimation, PillowWriter

    G, traces = sim.mall_graph, sim.traces
    if not traces: 
        raise ValueError("sim.traces is empty")
    if pos is None: 
        pos = nx.spring_layout(G, seed=sim.p.seed, k=0.8)

    cmap = {"anchor":"tab:red","similar":"tab:blue","different":"tab:green"}
    node_colors = [
        (cmap["anchor"] if G.nodes[n].get("role")=="anchor"
         else cmap.get(G.nodes[n].get("category"), "gray"))
        for n in G.nodes
    ]

    fig, ax = plt.subplots(figsize=(7,6)); ax.axis("off")
    nx.draw_networkx_edges(G, pos, ax=ax, alpha=.3, width=1.0)
    nx.draw_networkx_nodes(G, pos, ax=ax, node_color=node_colors, node_size=200)

    handles = [
        plt.Line2D([0],[0], marker="o", ls="", color=cmap["anchor"],   label="anchor"),
        plt.Line2D([0],[0], marker="o", ls="", color=cmap["similar"],  label="similar"),
        plt.Line2D([0],[0], marker="o", ls="", color=cmap["different"],label="different"),
        plt.Line2D([0],[0], marker="o", ls="", color="black",          label="agents"),
    ]
    ax.legend(handles=handles, loc="upper left", frameon=True)

    def XY(arr): 
        return [pos[int(n)][0] for n in arr], [pos[int(n)][1] for n in arr]
    x0, y0 = XY(traces[0])
    scat = ax.scatter(x0, y0, s=40, c="black", alpha=.85, zorder=10)
    ax.set_title("Agent movement (tick=0)")

    def update(t):
        x, y = XY(traces[t]); scat.set_offsets(np.c_[x, y])
        ax.set_title(f"Agent movement (tick={t})"); return scat,

    anim = FuncAnimation(fig, update, frames=len(traces), interval=interval, blit=False)
    ensure_dir("figs")
    writer = PillowWriter(fps=max(1, int(1000/interval)))
    anim.save(filename, writer=writer)
    plt.close(fig)
    print(f"[Saved GIF] {filename}")


In [28]:
params = Params(steps=50, n_agents=200, p_stay_anchor=0.2)
df, sim, A0 = simulate(params)

display(Markdown(f"**Anchor fixed at node:** `{A0}`"))
display(df.sort_values("footfall", ascending=False).head(10))

# footfall plot
plt.figure(figsize=(8,4))
plt.bar(df["node"], df["footfall"], color="skyblue")
plt.xlabel("Node"); plt.ylabel("Footfall")
plt.title("Footfall by Node — Step 2 (Visible)")
plt.tight_layout()
plt.show()

# animation
animate_movement(sim, filename="figs/step2_visible.gif", interval=params.interval_ms)
display(Image("figs/step2_visible.gif"))


In [29]:
import ipywidgets as w

theta = w.FloatSlider(value=0.6, min=0.0, max=1.0, step=0.05, description=r'θ_same')
pmove = w.FloatSlider(value=0.5, min=0.0, max=1.0, step=0.05, description='p_move_below')
pstaya = w.FloatSlider(value=0.2, min=0.0, max=0.8, step=0.05, description='p_stay_anchor')
runbtn = w.Button(description="Run Simulation", button_style="primary")
out = w.Output()

def rerun(_=None):
    with out:
        out.clear_output(wait=True)
        p = Params(steps=40, n_agents=180, theta_same=theta.value, p_move_below=pmove.value, p_stay_anchor=pstaya.value)
        df2, sim2, A02 = simulate(p)
        display(df2.sort_values("footfall", ascending=False).head(5))
        plt.figure(figsize=(8,4))
        plt.bar(df2["node"], df2["footfall"], color="lightcoral")
        plt.xlabel("Node"); plt.ylabel("Footfall"); plt.title("Footfall (interactive run)")
        plt.tight_layout(); plt.show()
        animate_movement(sim2, filename="figs/step2_visible.gif", interval=p.interval_ms)
        display(Image("figs/step2_visible.gif"))

runbtn.on_click(rerun)
display(w.VBox([theta, pmove, pstaya, runbtn, out]))

## **Visualization Adjustment**
Agents (black dots) were sometimes hidden under nodes.  
We changed the drawing order so agents are rendered **on the top layer** (`zorder=10`),  
making their movement clearly visible without modifying the core Step-2 logic.

---

## **Next Step**
In Step 3, “different” will be split into two explicit categories —  
`complementary` vs. `unrelated` — so we can model more realistic shopping behavior.
